# サイコロの目の和

In [10]:
from collections import defaultdict

#確率分布のdict作成
dist = {i:1/6 for i in range(1, 7)}

#2次元確率分布計算
dist_multi = defaultdict(lambda: 0)
for key1, v1 in dist.items():
    for key2, v2 in dist.items():
        dist_multi[key1 + key2] += v1*v2

#期待値計算
E = 0
for key ,v in dist_multi.items():
    E += key*v

print(E)

7.0


# 分布モデルとサンプルモデル

In [14]:
import numpy as np

def sample(dices=2):
    x = 0
    for _ in range(dices):
        x += np.random.choice([1, 2, 3, 4, 5, 6])
    return x

print(sample())

12


# モンテカルロ法の実装

In [16]:
#自分の実装

n = 1000

samples = np.array([sample() for i in range(n)])
E_sample = np.mean(samples)
print(E_sample)

7.008


In [19]:
#教科書の実装

trial = 1000
V, n = 0, 0

for _ in range(trial):
    s = sample()
    n += 1
    V += (s - V) / n
print(V)

7.007999999999997
